# Model Iterations:  Model 4

In [10]:
import os, shutil
import numpy as np
import random
np.random.seed(123)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time
import scipy
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.callbacks import EarlyStopping

parent_dir = '../../'

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import modelling as m

In [3]:
# plot params:
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['savefig.directory'] = parent_dir + '/figures/'

In [4]:
val_folder = parent_dir + 'data/seg_val/'
test_folder = parent_dir + 'data/seg_test/'
train_folder = parent_dir + 'data/seg_train/'

## Prepare for Modelling

First, we'll reshape the images in all the different datasets and create data generators for them.  I choose batch sizes that are 10% of the size of the datasets - as I did for model 2.

In [5]:
# test generator:
test_gen = ImageDataGenerator(rescale = 1. / 255).flow_from_directory(test_folder, batch_size = 300)

# train generator:
train_gen = ImageDataGenerator(rescale = 1. / 255).flow_from_directory(train_folder, batch_size = 1123)

# val generator:
val_gen = ImageDataGenerator(rescale = 1. / 255).flow_from_directory(val_folder, batch_size = 280)

Found 3000 images belonging to 6 classes.
Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.


In [6]:
# create the data sets
train_images, train_labels = next(train_gen)
test_images, test_labels = next(test_gen)
val_images, val_labels = next(val_gen)

Next, let's reshape in the same way that we did for model 2:

In [7]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

train_y = np.reshape(train_labels[:,0], (1123, 1))
test_y = np.reshape(test_labels[:,0], (300, 1))
val_y = np.reshape(val_labels[:,0], (280, 1))

In [11]:
model4 = Sequential()

model4.add(Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))

model4.add(Conv2D(180,kernel_size=(3,3),activation='relu'))
model4.add(MaxPooling2D(5,5))

model4.add(Conv2D(180,kernel_size=(3,3),activation='relu'))
model4.add(Conv2D(140,kernel_size=(3,3),activation='relu'))
model4.add(Conv2D(100,kernel_size=(3,3),activation='relu'))
model4.add(Conv2D(50,kernel_size=(3,3),activation='relu'))
model4.add(MaxPooling2D(5,5))

model4.add(Flatten())

model4.add(Dense(180,activation='relu'))
model4.add(Dense(100,activation='relu'))
model4.add(Dense(50,activation='relu'))

model4.add(Dropout(rate=0.5))

model4.add(Dense(6,activation='softmax'))

In [12]:
# compile
model4.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])

In [13]:
# view the summary:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 200)     5600      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 146, 146, 180)     324180    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 29, 29, 180)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 27, 180)       291780    
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 25, 25, 140)       226940    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 23, 23, 100)       126100    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 21, 21, 50)       

In [14]:
callback = EarlyStopping(patience = 3)
results4 = model4.fit(train_images, train_labels, epochs = 25, batch_size = 32, 
                      validation_data = (val_img, val_y), callbacks = [callback])

ValueError: Error when checking input: expected conv2d_9_input to have shape (150, 150, 3) but got array with shape (256, 256, 3)